In [2]:
%load_ext autoreload

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import protfasta 
import re

from Bio import pairwise2
from Bio.Seq import Seq 

%autoreload 2
%aimport AD_predictor_tools
%aimport AD_comparison_tools
%aimport PlottingTools

/opt/anaconda3/envs/jupyter_env/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [4]:
# Loading in this 
TF_prot_coords = pd.read_csv("../data/SFARI_TFs_with_knownADs_coords_ENSG.csv", index_col = 0)
TF_prot_coords

uniprotID  Start   End             ENSG
0     Q9HBZ2      1   717  ENSG00000172379
1     Q96QS3      1   562  ENSG00000004848
2     Q86V15      1  1759  ENSG00000130940
3     Q6P1N0      1   951  ENSG00000132024
4     O94983      1  1202  ENSG00000108509
5     O60479      1   287  ENSG00000064195
6     Q9H4W6      1   596  ENSG00000108001
7     Q06889      1   387  ENSG00000179388
8     Q92731      1   530  ENSG00000140009
9     O95718      1   433  ENSG00000119715
10    Q8NBF1      1   620  ENSG00000174332
11    P11308      1   479  ENSG00000157554
12    Q5T1R4      1  2406  ENSG00000127124
13    Q13422      1   519  ENSG00000185811
14    Q9UGL1      1  1544  ENSG00000117139
15    Q9BXK1      1   252  ENSG00000129911
16    O75840      1   302  ENSG00000118263
17    Q03164      1  3969  ENSG00000118058
18    Q06413      1   473  ENSG00000081189
19    O14770      1   477  ENSG00000134138
20    P35548      1   267  ENSG00000120149
21    Q14872      1   753  ENSG00000188786
22    Q9UL68      1  1186  ENSG00000186487
23    Q15788      1  1441  ENSG00000084676
24    Q9Y4A8      1   694  ENSG00000050344
25    Q12857      1   509  ENSG00000162599
26    Q14938      1   502  ENSG00000008441
27    P20393      1   614  ENSG00000126368
28    P43354      1   598  ENSG00000153234
29    O95096      1   273  ENSG00000125820
30    P32242      1   354  ENSG00000115507
31    Q02548      1   391  ENSG00000196092
32    P26367      1   422  ENSG00000007372
33    P78337      1   314  ENSG00000069011
34    P35398      1   523  ENSG00000069667
35    Q13485      1   552  ENSG00000141646
36    Q6ZRS2      1  3230  ENSG00000080603
37    Q9Y458      1   520  ENSG00000122145
38    Q9UGU0      1  1960  ENSG00000100207
39    P15884      1   667  ENSG00000196628
40    Q9NQB0      1   619  ENSG00000148737
41    Q6N021      1  2002  ENSG00000168769
42    P19532      1   575  ENSG00000068323
43    P10827      1   490  ENSG00000126351
44    P25490      1   414  ENSG00000100811
45    P11473      1   427  ENSG00000111424

In [5]:
for uniprotID in TF_prot_coords["uniprotID"]:
    print(uniprotID)

Q9HBZ2
Q96QS3
Q86V15
Q6P1N0
O94983
O60479
Q9H4W6
Q06889
Q92731
O95718
Q8NBF1
P11308
Q5T1R4
Q13422
Q9UGL1
Q9BXK1
O75840
Q03164
Q06413
O14770
P35548
Q14872
Q9UL68
Q15788
Q9Y4A8
Q12857
Q14938
P20393
P43354
O95096
P32242
Q02548
P26367
P78337
P35398
Q13485
Q6ZRS2
Q9Y458
Q9UGU0
P15884
Q9NQB0
Q6N021
P19532
P10827
P25490
P11473


In [6]:
biomart_output = pd.read_csv('../data/TF_CDS_biomart_output.txt')
# biomart_output = biomart_output[["Chromosome/scaffold name", "Genomic coding start", "Genomic coding end", "Gene stable ID", "Transcript stable ID", "Strand"]]
biomart_output = biomart_output.dropna()
biomart_output["Genomic coding start"] -= 1 # because BED format
biomart_output["Genomic coding start"] = biomart_output["Genomic coding start"].astype(int)
biomart_output["Genomic coding end"] = biomart_output["Genomic coding end"].astype(int)

In [7]:
biomart_output["Strand"] = biomart_output["Strand"].map({-1 : "-", +1 : "+"})
biomart_output

Gene stable ID  Genomic coding start  Genomic coding end Strand  \
0     ENSG00000134138              37099454            37099466      -   
1     ENSG00000134138              37097966            37098199      -   
2     ENSG00000134138              37096288            37096430      -   
3     ENSG00000134138              37095563            37095614      -   
4     ENSG00000134138              37094526            37094577      -   
...               ...                   ...                 ...    ...   
2457  ENSG00000277800                 74638               74813      -   
2458  ENSG00000050344              26152498            26153068      +   
2459  ENSG00000050344              26177942            26178122      +   
2460  ENSG00000050344              26183700            26183784      +   
2461  ENSG00000050344              26184532            26185783      +   

     Transcript stable ID Chromosome/scaffold name  
0         ENST00000561208                       15  
1         ENST00000561208                       15  
2         ENST00000561208                       15  
3         ENST00000561208                       15  
4         ENST00000561208                       15  
...                   ...                      ...  
2457      ENST00000619509           HSCHRX_2_CTG12  
2458      ENST00000056233                        7  
2459      ENST00000056233                        7  
2460      ENST00000056233                        7  
2461      ENST00000056233                        7  

[2324 rows x 6 columns]

In [8]:
biomart_output[biomart_output["Gene stable ID"] == "ENSG00000157554"]

Gene stable ID  Genomic coding start  Genomic coding end Strand  \
303  ENSG00000157554              38403505            38403709      -   
304  ENSG00000157554              38402556            38402637      -   
305  ENSG00000157554              38392375            38392444      -   
306  ENSG00000157554              38391658            38391715      -   
307  ENSG00000157554              38390994            38391042      -   
308  ENSG00000157554              38383402            38383923      -   
310  ENSG00000157554              38423409            38423521      -   
311  ENSG00000157554              38445403            38445621      -   
312  ENSG00000157554              38423409            38423561      -   
313  ENSG00000157554              38403505            38403709      -   
314  ENSG00000157554              38402556            38402637      -   
315  ENSG00000157554              38392375            38392444      -   
316  ENSG00000157554              38391658            38391715      -   
317  ENSG00000157554              38390994            38391042      -   
318  ENSG00000157554              38383402            38383923      -   
320  ENSG00000157554              38575661            38575700      -   
321  ENSG00000157554              38445403            38445621      -   
322  ENSG00000157554              38423409            38423561      -   
323  ENSG00000157554              38403505            38403709      -   
324  ENSG00000157554              38402556            38402637      -   
325  ENSG00000157554              38392375            38392444      -   
326  ENSG00000157554              38391658            38391715      -   
327  ENSG00000157554              38390994            38391042      -   
328  ENSG00000157554              38383402            38383923      -   
329  ENSG00000157554              38400573            38400645      -   
331  ENSG00000157554              38575661            38575700      -   
332  ENSG00000157554              38445403            38445621      -   
333  ENSG00000157554              38423409            38423561      -   
334  ENSG00000157554              38403505            38403709      -   
335  ENSG00000157554              38402556            38402637      -   
336  ENSG00000157554              38392375            38392444      -   
337  ENSG00000157554              38391658            38391715      -   
338  ENSG00000157554              38390994            38391042      -   
339  ENSG00000157554              38400573            38400645      -   
340  ENSG00000157554              38498362            38498380      -   
341  ENSG00000157554              38383402            38383923      -   
342  ENSG00000157554              38445403            38445621      -   
343  ENSG00000157554              38423409            38423561      -   
344  ENSG00000157554              38403505            38403709      -   
345  ENSG00000157554              38402556            38402637      -   
346  ENSG00000157554              38392375            38392444      -   
347  ENSG00000157554              38391658            38391715      -   
348  ENSG00000157554              38390994            38391042      -   
349  ENSG00000157554              38400573            38400645      -   
351  ENSG00000157554              38575661            38575700      -   
352  ENSG00000157554              38383402            38383923      -   
353  ENSG00000157554              38445403            38445621      -   
354  ENSG00000157554              38423409            38423561      -   
355  ENSG00000157554              38403505            38403709      -   
356  ENSG00000157554              38402556            38402637      -   
357  ENSG00000157554              38392375            38392444      -   
358  ENSG00000157554              38391658            38391715      -   
359  ENSG00000157554              38390994            38391042      -   
361  ENSG00000157554              38575661            38575700  

In [9]:
len(set(biomart_output['Gene stable ID']))

50

In [10]:
len(TF_prot_coords["uniprotID"])

46

In [11]:
# Matches whats expected
set(TF_prot_coords["ENSG"]) - set(biomart_output['Gene stable ID'])

set()

In [12]:
set(biomart_output['Gene stable ID']) - set(TF_prot_coords["ENSG"])

{'ENSG00000262024', 'ENSG00000276461', 'ENSG00000277800', 'ENSG00000288293'}

In [13]:
uniprotID_ENST_mapping = pd.read_csv("../data/SFARI_TFs_with_ENST_corrected.csv")
uniprotID_ENST_mapping = uniprotID_ENST_mapping[["uniprotID", "ENST"]]
uniprotID_ENST_mapping["ENST"] = uniprotID_ENST_mapping["ENST"].str.split(".").str[0]
uniprotID_ENST_mapping_dict= dict(zip(uniprotID_ENST_mapping["uniprotID"], uniprotID_ENST_mapping["ENST"]))
# New ones after redone with ADs I found in canonical
uniprotID_ENST_mapping_dict['O60479'] = 'ENST00000434704'
# uniprotID_ENST_mapping_dict['O95718'] = 
# I couldn't find a ENST code corresponding to O95718

In [14]:
# Saving bed files now
for i in TF_prot_coords.index:
    ENSG = TF_prot_coords["ENSG"].loc[i]
    uniprotID = TF_prot_coords["uniprotID"].loc[i]
    if uniprotID == "O95718":
        continue
    ENST = uniprotID_ENST_mapping_dict[uniprotID]
    biomart_output_for_ENSG = biomart_output[biomart_output["Gene stable ID"] == ENSG]
    biomart_output_for_ENSG = biomart_output_for_ENSG[biomart_output_for_ENSG["Transcript stable ID"] == ENST]
    print(uniprotID)
    
    biomart_output_for_ENSG["uniprotID"] = uniprotID
    biomart_output_for_ENSG = biomart_output_for_ENSG[["Chromosome/scaffold name", "Genomic coding start", 
                                                       "Genomic coding end", "uniprotID", "Transcript stable ID", "Strand"]]
    display(biomart_output_for_ENSG)

    filepath = "../soto_analysis/outputs/mutations/cds_bed_format/" + ENST
    biomart_output_for_ENSG.to_csv(filepath, index = False, sep = '\t', header = None)


#12	47878967	47879113	P11473	ENST00000229022	-


Q9HBZ2


Chromosome/scaffold name  Genomic coding start  Genomic coding end  \
2037                       15              80404515            80404546   
2038                       15              80450879            80450994   
2039                       15              80457928            80457976   
2040                       15              80470217            80470431   
2041                       15              80475009            80475223   
2042                       15              80508155            80508258   
2043                       15              80513910            80513976   
2044                       15              80514319            80514405   
2045                       15              80551198            80551275   
2046                       15              80552639            80552774   
2047                       15              80555064            80555139   
2048                       15              80563087            80563239   
2049                       15              80574147            80574220   
2050                       15              80574986            80575110   
2051                       15              80576865            80576965   
2052                       15              80580410            80580549   
2053                       15              80581238            80581404   
2054                       15              80591567            80591704   
2055                       15              80593599            80593698   

     uniprotID Transcript stable ID Strand  
2037    Q9HBZ2      ENST00000303329      +  
2038    Q9HBZ2      ENST00000303329      +  
2039    Q9HBZ2      ENST00000303329      +  
2040    Q9HBZ2      ENST00000303329      +  
2041    Q9HBZ2      ENST00000303329      +  
2042    Q9HBZ2      ENST00000303329      +  
2043    Q9HBZ2      ENST00000303329      +  
2044    Q9HBZ2      ENST00000303329      +  
2045    Q9HBZ2      ENST00000303329      +  
2046    Q9HBZ2      ENST00000303329      +  
2047    Q9HBZ2      ENST00000303329      +  
2048    Q9HBZ2      ENST00000303329      +  
2049    Q9HBZ2      ENST00000303329      +  
2050    Q9HBZ2      ENST00000303329      +  
2051    Q9HBZ2      ENST00000303329      +  
2052    Q9HBZ2      ENST00000303329      +  
2053    Q9HBZ2      ENST00000303329      +  
2054    Q9HBZ2      ENST00000303329      +  
2055    Q9HBZ2      ENST00000303329      +

Q96QS3


Chromosome/scaffold name  Genomic coding start  Genomic coding end  \
1991                        X              25015541            25015737   
1992                        X              25012921            25013798   
1993                        X              25010259            25010305   
1994                        X              25007110            25007439   
1995                        X              25004669            25004910   

     uniprotID Transcript stable ID Strand  
1991    Q96QS3      ENST00000379044      -  
1992    Q96QS3      ENST00000379044      -  
1993    Q96QS3      ENST00000379044      -  
1994    Q96QS3      ENST00000379044      -  
1995    Q96QS3      ENST00000379044      -

Q86V15


...

Chromosome/scaffold name  Genomic coding start  Genomic coding end  \
275                       17              40074488            40074541   
276                       17              40076870            40076938   
277                       17              40077507            40077608   
278                       17              40083834            40083982   
279                       17              40084609            40084815   
280                       17              40086706            40086853   
281                       17              40088241            40088500   
282                       17              40089205            40089333   
284                       17              40093019            40093382   

    uniprotID Transcript stable ID Strand  
275    P10827      ENST00000264637      +  
276    P10827      ENST00000264637      +  
277    P10827      ENST00000264637      +  
278    P10827      ENST00000264637      +  
279    P10827      ENST00000264637      +  
280    P10827      ENST00000264637      +  
281    P10827      ENST00000264637      +  
282    P10827      ENST00000264637      +  
284    P10827      ENST00000264637      +

P25490


Chromosome/scaffold name  Genomic coding start  Genomic coding end  \
789                       14             100239244           100239923   
790                       14             100262303           100262466   
791                       14             100274697           100274758   
792                       14             100276489           100276648   
793                       14             100277417           100277600   

    uniprotID Transcript stable ID Strand  
789    P25490      ENST00000262238      +  
790    P25490      ENST00000262238      +  
791    P25490      ENST00000262238      +  
792    P25490      ENST00000262238      +  
793    P25490      ENST00000262238      +

P11473


Chromosome/scaffold name  Genomic coding start  Genomic coding end  \
373                       12              47878967            47879113   
374                       12              47865046            47865177   
375                       12              47857503            47857688   
376                       12              47857128            47857249   
377                       12              47855629            47855801   
378                       12              47846656            47846808   
379                       12              47846334            47846451   
380                       12              47844745            47845005   

    uniprotID Transcript stable ID Strand  
373    P11473      ENST00000395324      -  
374    P11473      ENST00000395324      -  
375    P11473      ENST00000395324      -  
376    P11473      ENST00000395324      -  
377    P11473      ENST00000395324      -  
378    P11473      ENST00000395324      -  
379    P11473      ENST00000395324      -  
380    P11473      ENST00000395324      -

In [15]:
# Now, preparing the format of the domain input to get_DBD_ED_coords.py

In [16]:
AD_coords = pd.read_csv("../data/SFARI_ADs_AA_coords_ENSG.csv", index_col = 0)
AD_coords

uniprotID  Start  End             ENSG
0     Q9HBZ2    524  717  ENSG00000172379
1     Q96QS3    472  562  ENSG00000004848
2     Q6P1N0     22   60  ENSG00000132024
3     O94983    285  468  ENSG00000108509
4     O94983    472  581  ENSG00000108509
..       ...    ...  ...              ...
58    P10827      1   52  ENSG00000126351
59    P11473    415  427  ENSG00000111424
60    P11473    195  238  ENSG00000111424
61    P25490      1   69  ENSG00000100811
62    P25490    371  397  ENSG00000100811

[63 rows x 4 columns]

In [17]:
AD_coords["coord"] = AD_coords["Start"].astype(str)+"-"+AD_coords["End"].astype(str)
AD_coords

uniprotID  Start  End             ENSG    coord
0     Q9HBZ2    524  717  ENSG00000172379  524-717
1     Q96QS3    472  562  ENSG00000004848  472-562
2     Q6P1N0     22   60  ENSG00000132024    22-60
3     O94983    285  468  ENSG00000108509  285-468
4     O94983    472  581  ENSG00000108509  472-581
..       ...    ...  ...              ...      ...
58    P10827      1   52  ENSG00000126351     1-52
59    P11473    415  427  ENSG00000111424  415-427
60    P11473    195  238  ENSG00000111424  195-238
61    P25490      1   69  ENSG00000100811     1-69
62    P25490    371  397  ENSG00000100811  371-397

[63 rows x 5 columns]

In [18]:
merged_AD_coords = AD_coords[["uniprotID", "coord"]].groupby("uniprotID").agg(lambda c: ",".join(c))
merged_AD_coords = merged_AD_coords.drop(["O95718"])
merged_AD_coords = merged_AD_coords.reset_index()
merged_AD_coords

uniprotID                    coord
0     O14770                  340-477
1     O60479             199-263,2-91
2     O75840                    2-101
3     O94983          285-468,472-581
4     O95096                  220-273
5     P10827                     1-52
6     P11308          433-479,118-261
7     P11473          415-427,195-238
8     P15884            1-100,340-400
9     P19532    212-333,472-575,1-127
10    P20393                    8-285
11    P25490             1-69,371-397
12    P26367                  271-422
13    P32242                  172-354
14    P35398                  272-385
15    P35548                  234-267
16    P43354             1-91,584-598
17    P78337                  234-283
18    Q02548                  304-358
19    Q03164                2782-2921
20    Q06413                   87-473
21    Q06889                   13-214
22    Q12857                  427-509
23    Q13422                  284-365
24    Q13485            30-60,275-552
25    Q14872          510-624,328-509
26    Q14938                  414-502
27    Q15788  1-93,1241-1385,840-1011
28    Q5T1R4                 211-1074
29    Q6N021                1582-1751
30    Q6P1N0                    22-60
31    Q6ZRS2       2575-3230,198-1445
32    Q86V15                   31-185
33    Q8NBF1                  447-620
34    Q92731            1-148,304-500
35    Q96QS3                  472-562
36    Q9BXK1                  209-252
37    Q9H4W6                  281-360
38    Q9HBZ2                  524-717
39    Q9NQB0             1-53,459-505
40    Q9UGL1                1302-1391
41    Q9UGU0                    1-327
42    Q9UL68                  152-425
43    Q9Y458                  402-481
44    Q9Y4A8                  252-451

In [19]:
TF_table = merged_AD_coords.rename(columns = {"coord" : "AD_coords"})
TF_table = pd.merge(TF_table, AD_coords[["uniprotID", "ENSG"]], on = "uniprotID", how = "left").drop_duplicates()
TF_table = TF_table.reset_index(drop = True)
TF_table

uniprotID                AD_coords             ENSG
0     O14770                  340-477  ENSG00000134138
1     O60479             199-263,2-91  ENSG00000064195
2     O75840                    2-101  ENSG00000118263
3     O94983          285-468,472-581  ENSG00000108509
4     O95096                  220-273  ENSG00000125820
5     P10827                     1-52  ENSG00000126351
6     P11308          433-479,118-261  ENSG00000157554
7     P11473          415-427,195-238  ENSG00000111424
8     P15884            1-100,340-400  ENSG00000196628
9     P19532    212-333,472-575,1-127  ENSG00000068323
10    P20393                    8-285  ENSG00000126368
11    P25490             1-69,371-397  ENSG00000100811
12    P26367                  271-422  ENSG00000007372
13    P32242                  172-354  ENSG00000115507
14    P35398                  272-385  ENSG00000069667
15    P35548                  234-267  ENSG00000120149
16    P43354             1-91,584-598  ENSG00000153234
17    P78337                  234-283  ENSG00000069011
18    Q02548                  304-358  ENSG00000196092
19    Q03164                2782-2921  ENSG00000118058
20    Q06413                   87-473  ENSG00000081189
21    Q06889                   13-214  ENSG00000179388
22    Q12857                  427-509  ENSG00000162599
23    Q13422                  284-365  ENSG00000185811
24    Q13485            30-60,275-552  ENSG00000141646
25    Q14872          510-624,328-509  ENSG00000188786
26    Q14938                  414-502  ENSG00000008441
27    Q15788  1-93,1241-1385,840-1011  ENSG00000084676
28    Q5T1R4                 211-1074  ENSG00000127124
29    Q6N021                1582-1751  ENSG00000168769
30    Q6P1N0                    22-60  ENSG00000132024
31    Q6ZRS2       2575-3230,198-1445  ENSG00000080603
32    Q86V15                   31-185  ENSG00000130940
33    Q8NBF1                  447-620  ENSG00000174332
34    Q92731            1-148,304-500  ENSG00000140009
35    Q96QS3                  472-562  ENSG00000004848
36    Q9BXK1                  209-252  ENSG00000129911
37    Q9H4W6                  281-360  ENSG00000108001
38    Q9HBZ2                  524-717  ENSG00000172379
39    Q9NQB0             1-53,459-505  ENSG00000148737
40    Q9UGL1                1302-1391  ENSG00000117139
41    Q9UGU0                    1-327  ENSG00000100207
42    Q9UL68                  152-425  ENSG00000186487
43    Q9Y458                  402-481  ENSG00000122145
44    Q9Y4A8                  252-451  ENSG00000050344

In [20]:
TF_table["ENST"] = TF_table["uniprotID"].map(uniprotID_ENST_mapping_dict)
TF_table["DBD_coords"] = "NA-NA"
TF_table["RD_coords"] = ""
TF_table["Bif_coords"] = ""
TF_table["1"] = ""
TF_table["2"] = ""
TF_table["length"] = "1"
TF_table

uniprotID                AD_coords             ENSG             ENST  \
0     O14770                  340-477  ENSG00000134138  ENST00000561208   
1     O60479             199-263,2-91  ENSG00000064195  ENST00000434704   
2     O75840                    2-101  ENSG00000118263  ENST00000309446   
3     O94983          285-468,472-581  ENSG00000108509  ENST00000348066   
4     O95096                  220-273  ENSG00000125820  ENST00000377142   
5     P10827                     1-52  ENSG00000126351  ENST00000264637   
6     P11308          433-479,118-261  ENSG00000157554  ENST00000288319   
7     P11473          415-427,195-238  ENSG00000111424  ENST00000395324   
8     P15884            1-100,340-400  ENSG00000196628  ENST00000356073   
9     P19532    212-333,472-575,1-127  ENSG00000068323  ENST00000315869   
10    P20393                    8-285  ENSG00000126368  ENST00000246672   
11    P25490             1-69,371-397  ENSG00000100811  ENST00000262238   
12    P26367                  271-422  ENSG00000007372  ENST00000241001   
13    P32242                  172-354  ENSG00000115507  ENST00000282549   
14    P35398                  272-385  ENSG00000069667  ENST00000335670   
15    P35548                  234-267  ENSG00000120149  ENST00000239243   
16    P43354             1-91,584-598  ENSG00000153234  ENST00000339562   
17    P78337                  234-283  ENSG00000069011  ENST00000265340   
18    Q02548                  304-358  ENSG00000196092  ENST00000358127   
19    Q03164                2782-2921  ENSG00000118058  ENST00000389506   
20    Q06413                   87-473  ENSG00000081189  ENST00000437473   
21    Q06889                   13-214  ENSG00000179388  ENST00000317216   
22    Q12857                  427-509  ENSG00000162599  ENST00000403491   
23    Q13422                  284-365  ENSG00000185811  ENST00000331340   
24    Q13485            30-60,275-552  ENSG00000141646  ENST00000342988   
25    Q14872          510-624,328-509  ENSG00000188786  ENST00000373036   
26    Q14938                  414-502  ENSG00000008441  ENST00000592199   
27    Q15788  1-93,1241-1385,840-1011  ENSG00000084676  ENST00000348332   
28    Q5T1R4                 211-1074  ENSG00000127124  ENST00000372583   
29    Q6N021                1582-1751  ENSG00000168769  ENST00000380013   
30    Q6P1N0                    22-60  ENSG00000132024  ENST00000318003   
31    Q6ZRS2       2575-3230,198-1445  ENSG00000080603  ENST00000262518   
32    Q86V15                   31-185  ENSG00000130940  ENST00000377022   
33    Q8NBF1                  447-620  ENSG00000174332  ENST00000312233   
34    Q92731            1-148,304-500  ENSG00000140009  ENST00000341099   
35    Q96QS3                  472-562  ENSG00000004848  ENST00000379044   
36    Q9BXK1                  209-252  ENSG00000129911  ENST00000250916   
37    Q9H4W6                  281-360  ENSG00000108001  ENST00000355311   
38    Q9HBZ2                  524-717  ENSG00000172379  ENST00000303329   
39    Q9NQB0             1-53,459-505  ENSG00000148737  ENST00000355995   
40    Q9UGL1                1302-1391  ENSG00000117139  ENST00000367265   
41    Q9UGU0                    1-327  ENSG00000100207  ENST00000359486   
42    Q9UL68                  152-425  ENSG00000186487  ENST00000428368   
43    Q9Y458                  402-481  ENSG00000122145  ENST00000373294   
44    Q9Y4A8                  252-451  ENSG00000050344  ENST00000056233   

   DBD_coords RD_coords Bif_coords 1 2 length  
0       NA-NA                               1  
1       NA-NA                               1  
2       NA-NA                               1  
3       NA-NA                               1  
4       NA-NA                               1  
5       NA-NA                               1  
6       NA-NA                               1  
7       NA-NA                               1  
8       NA-NA                               1  
9       NA-NA                               1  
10      NA-NA           

Format of text table seems to be 

"../outputs/TFs_table_proteins.txt"

- 0
- 1
- 2
- 3 uniprotID
- 4 ensg
- 5 enst
- 6 dbd_coords 
- 7 ad_coords
- 8 rd_coords
- 9 bif_coords
- -1 len

the coords look like 1-3,4-5,6-8 etc.

In [21]:
TF_table_formatted = TF_table[["1","2", "uniprotID", "ENSG", "ENST", "DBD_coords", "AD_coords", "RD_coords", "Bif_coords", "length"]]
TF_table_formatted

1 2 uniprotID             ENSG             ENST DBD_coords  \
0         O14770  ENSG00000134138  ENST00000561208      NA-NA   
1         O60479  ENSG00000064195  ENST00000434704      NA-NA   
2         O75840  ENSG00000118263  ENST00000309446      NA-NA   
3         O94983  ENSG00000108509  ENST00000348066      NA-NA   
4         O95096  ENSG00000125820  ENST00000377142      NA-NA   
5         P10827  ENSG00000126351  ENST00000264637      NA-NA   
6         P11308  ENSG00000157554  ENST00000288319      NA-NA   
7         P11473  ENSG00000111424  ENST00000395324      NA-NA   
8         P15884  ENSG00000196628  ENST00000356073      NA-NA   
9         P19532  ENSG00000068323  ENST00000315869      NA-NA   
10        P20393  ENSG00000126368  ENST00000246672      NA-NA   
11        P25490  ENSG00000100811  ENST00000262238      NA-NA   
12        P26367  ENSG00000007372  ENST00000241001      NA-NA   
13        P32242  ENSG00000115507  ENST00000282549      NA-NA   
14        P35398  ENSG00000069667  ENST00000335670      NA-NA   
15        P35548  ENSG00000120149  ENST00000239243      NA-NA   
16        P43354  ENSG00000153234  ENST00000339562      NA-NA   
17        P78337  ENSG00000069011  ENST00000265340      NA-NA   
18        Q02548  ENSG00000196092  ENST00000358127      NA-NA   
19        Q03164  ENSG00000118058  ENST00000389506      NA-NA   
20        Q06413  ENSG00000081189  ENST00000437473      NA-NA   
21        Q06889  ENSG00000179388  ENST00000317216      NA-NA   
22        Q12857  ENSG00000162599  ENST00000403491      NA-NA   
23        Q13422  ENSG00000185811  ENST00000331340      NA-NA   
24        Q13485  ENSG00000141646  ENST00000342988      NA-NA   
25        Q14872  ENSG00000188786  ENST00000373036      NA-NA   
26        Q14938  ENSG00000008441  ENST00000592199      NA-NA   
27        Q15788  ENSG00000084676  ENST00000348332      NA-NA   
28        Q5T1R4  ENSG00000127124  ENST00000372583      NA-NA   
29        Q6N021  ENSG00000168769  ENST00000380013      NA-NA   
30        Q6P1N0  ENSG00000132024  ENST00000318003      NA-NA   
31        Q6ZRS2  ENSG00000080603  ENST00000262518      NA-NA   
32        Q86V15  ENSG00000130940  ENST00000377022      NA-NA   
33        Q8NBF1  ENSG00000174332  ENST00000312233      NA-NA   
34        Q92731  ENSG00000140009  ENST00000341099      NA-NA   
35        Q96QS3  ENSG00000004848  ENST00000379044      NA-NA   
36        Q9BXK1  ENSG00000129911  ENST00000250916      NA-NA   
37        Q9H4W6  ENSG00000108001  ENST00000355311      NA-NA   
38        Q9HBZ2  ENSG00000172379  ENST00000303329      NA-NA   
39        Q9NQB0  ENSG00000148737  ENST00000355995      NA-NA   
40        Q9UGL1  ENSG00000117139  ENST00000367265      NA-NA   
41        Q9UGU0  ENSG00000100207  ENST00000359486      NA-NA   
42        Q9UL68  ENSG00000186487  ENST00000428368      NA-NA   
43        Q9Y458  ENSG00000122145  ENST00000373294      NA-NA   
44        Q9Y4A8  ENSG00000050344  ENST00000056233      NA-NA   

                  AD_coords RD_coords Bif_coords length  
0                   340-477                           1  
1              199-263,2-91                           1  
2                     2-101                           1  
3           285-468,472-581                           1  
4                   220-273                           1  
5                      1-52                           1  
6           433-479,118-261                           1  
7           415-427,195-238                           1  
8             1-100,340-400                           1  
9     212-333,472-575,1-127                           1  
10                    8-285                           1  
11             1-69,371-397                           1  
12                  271-422                           1  
13                  172-354                           1  
14                  272-385                           1  
15                  234-267                           1  
16             1-91,584-59

In [22]:
TF_table_formatted.to_csv("../soto_analysis/outputs/TFs_table_proteins.txt", sep = "\t")

In [23]:
TF_table_formatted[["uniprotID"]].to_csv("../data/TF_table_uniprotIDs.txt",
                               header=None, index=None, sep=' ', mode='a')

In [24]:
uniprot_DBD_output = pd.read_csv("../data/TF_table_DBDs.txt", sep = "\t")
uniprot_DBD_output["DBD_split"] = uniprot_DBD_output["DNA binding"].str.split("DNA_bind")
uniprot_DBD_output

From   Entry                                        DNA binding  \
0   O14770  O14770  DNA_BIND 276..338; /note="Homeobox; TALE-type"...   
1   O60479  O60479  DNA_BIND 129..188; /note="Homeobox"; /evidence...   
2   O75840  O75840                                                NaN   
3   O94983  O94983  DNA_BIND 30..155; /note="CG-1"; /evidence="ECO...   
4   O95096  O95096  DNA_BIND 128..187; /note="Homeobox"; /evidence...   
5   P10827  P10827  DNA_BIND 53..127; /note="Nuclear receptor"; /e...   
6   P11308  P11308  DNA_BIND 311..391; /note="ETS"; /evidence="ECO...   
7   P11473  P11473  DNA_BIND 21..96; /note="Nuclear receptor"; /ev...   
8   P15884  P15884                                                NaN   
9   P19532  P19532                                                NaN   
10  P20393  P20393  DNA_BIND 129..205; /note="Nuclear receptor"; /...   
11  P25490  P25490                                                NaN   
12  P26367  P26367  DNA_BIND 4..130; /note="Paired"; /evidence="EC...   
13  P32242  P32242  DNA_BIND 38..97; /note="Homeobox"; /evidence="...   
14  P35398  P35398  DNA_BIND 73..138; /note="Nuclear receptor"; /e...   
15  P35548  P35548  DNA_BIND 142..201; /note="Homeobox"; /evidence...   
16  P43354  P43354  DNA_BIND 260..335; /note="Nuclear receptor"; /...   
17  P78337  P78337  DNA_BIND 89..148; /note="Homeobox"; /evidence=...   
18  Q02548  Q02548  DNA_BIND 16..142; /note="Paired"; /evidence="E...   
19  Q03164  Q03164  DNA_BIND 169..180; /note="A.T hook 1"; DNA_BIN...   
20  Q06413  Q06413  DNA_BIND 58..86; /note="Mef2-type"; /evidence=...   
21  Q06889  Q06889                                                NaN   
22  Q12857  Q12857  DNA_BIND 1..194; /note="CTF/NF-I"; /evidence="...   
23  Q13422  Q13422                                                NaN   
24  Q13485  Q13485                                                NaN   
25  Q14872  Q14872                                                NaN   
26  Q14938  Q14938  DNA_BIND 1..194; /note="CTF/NF-I"; /evidence="...   
27  Q15788  Q15788                                                NaN   
28  Q5T1R4  Q5T1R4                                                NaN   
29  Q6N021  Q6N021                                                NaN   
30  Q6P1N0  Q6P1N0                                                NaN   
31  Q6ZRS2  Q6ZRS2  DNA_BIND 2857..2869; /note="A.T hook 1"; DNA_B...   
32  Q86V15  Q86V15                                                NaN   
33  Q8NBF1  Q8NBF1                                                NaN   
34  Q92731  Q92731  DNA_BIND 149..214; /note="Nuclear receptor"; /...   
35  Q96QS3  Q96QS3  DNA_BIND 328..387; /note="Homeobox"; /evidence...   
36  Q9BXK1  Q9BXK1                                                NaN   
37  Q9H4W6  Q9H4W6                                                NaN   
38  Q9HBZ2  Q9HBZ2                                                NaN   
39  Q9NQB0  Q9NQB0  DNA_BIND 350..418; /note="HMG box"; /evidence=...   
40  Q9UGL1  Q9UGL1                                                NaN   
41  Q9UGU0  Q9UGU0              DNA_BIND 1537..1551; /note="A.T hook"   
42  Q9UL68  Q9UL68                                                NaN   
43  Q9Y458  Q9Y458  DNA_BIND 96..283; /note="T-box"; /evidence="EC...   
44  Q9Y4A8  Q9Y4A8                                                NaN   

                                            DBD_split  
0   [DNA_BIND 276..338; /note="Homeobox; TALE-type...  
1   [DNA_BIND 129..188; /note="Homeobox"; /evidenc...  
2                                                 NaN  
3   [DNA_BIND 30..155; /note="CG-1"; /evidence="EC...  
4   [DNA_BIND 128..187; /note="Homeobox"; /evidenc...  
5   [DNA_BIND 53..127; /note="Nuclear receptor"; /...  
6   [DNA_BIND 311..391; /note="ETS"; /evidence="EC...  
7   [DNA_BIND 21..96; /note="Nuclear receptor"; /e...  
8                                                 NaN  
9                                                 NaN  
10  [DNA_BIND 129..205; /note="

In [25]:
DBDs = uniprot_DBD_output.explode("DBD_split")
DBDs = DBDs.dropna()
DBDs["Start"] = DBDs["DBD_split"].str.extract(r'DNA_BIND (\d*)\.\.')
DBDs["End"] = DBDs["DBD_split"].str.extract(r'DNA_BIND \d*\.\.(\d*)')
DBDs = DBDs[["From", "Start", "End"]]
DBDs = DBDs.rename(columns = {"From" : "uniprotID"})
DBDs["Start"] = DBDs["Start"].astype(int)
DBDs["End"] = DBDs["End"].astype(int)
DBDs["coord"] = DBDs["Start"].astype(str)+"-"+DBDs["End"].astype(str)
DBDs

uniprotID  Start   End      coord
0     O14770    276   338    276-338
1     O60479    129   188    129-188
3     O94983     30   155     30-155
4     O95096    128   187    128-187
5     P10827     53   127     53-127
6     P11308    311   391    311-391
7     P11473     21    96      21-96
10    P20393    129   205    129-205
12    P26367      4   130      4-130
13    P32242     38    97      38-97
14    P35398     73   138     73-138
15    P35548    142   201    142-201
16    P43354    260   335    260-335
17    P78337     89   148     89-148
18    Q02548     16   142     16-142
19    Q03164    169   180    169-180
20    Q06413     58    86      58-86
22    Q12857      1   194      1-194
26    Q14938      1   194      1-194
31    Q6ZRS2   2857  2869  2857-2869
34    Q92731    149   214    149-214
35    Q96QS3    328   387    328-387
39    Q9NQB0    350   418    350-418
41    Q9UGU0   1537  1551  1537-1551
43    Q9Y458     96   283     96-283

In [26]:
merged_DBD_coords = DBDs[["uniprotID", "coord"]].groupby("uniprotID").agg(lambda c: ",".join(c))
# merged_DBD_coords = merged_DBD_coords.drop(["O95718"])
merged_DBD_coords = merged_DBD_coords.reset_index()
merged_DBD_coords

uniprotID      coord
0     O14770    276-338
1     O60479    129-188
2     O94983     30-155
3     O95096    128-187
4     P10827     53-127
5     P11308    311-391
6     P11473      21-96
7     P20393    129-205
8     P26367      4-130
9     P32242      38-97
10    P35398     73-138
11    P35548    142-201
12    P43354    260-335
13    P78337     89-148
14    Q02548     16-142
15    Q03164    169-180
16    Q06413      58-86
17    Q12857      1-194
18    Q14938      1-194
19    Q6ZRS2  2857-2869
20    Q92731    149-214
21    Q96QS3    328-387
22    Q9NQB0    350-418
23    Q9UGU0  1537-1551
24    Q9Y458     96-283

In [27]:
TF_table_formatted = pd.merge(TF_table_formatted, merged_DBD_coords, how = "left", on = "uniprotID")
TF_table_formatted["DBD_coords"] = TF_table_formatted["coord"]
TF_table_formatted = TF_table_formatted.drop(columns = ["coord"])
TF_table_formatted

1 2 uniprotID             ENSG             ENST DBD_coords  \
0         O14770  ENSG00000134138  ENST00000561208    276-338   
1         O60479  ENSG00000064195  ENST00000434704    129-188   
2         O75840  ENSG00000118263  ENST00000309446        NaN   
3         O94983  ENSG00000108509  ENST00000348066     30-155   
4         O95096  ENSG00000125820  ENST00000377142    128-187   
5         P10827  ENSG00000126351  ENST00000264637     53-127   
6         P11308  ENSG00000157554  ENST00000288319    311-391   
7         P11473  ENSG00000111424  ENST00000395324      21-96   
8         P15884  ENSG00000196628  ENST00000356073        NaN   
9         P19532  ENSG00000068323  ENST00000315869        NaN   
10        P20393  ENSG00000126368  ENST00000246672    129-205   
11        P25490  ENSG00000100811  ENST00000262238        NaN   
12        P26367  ENSG00000007372  ENST00000241001      4-130   
13        P32242  ENSG00000115507  ENST00000282549      38-97   
14        P35398  ENSG00000069667  ENST00000335670     73-138   
15        P35548  ENSG00000120149  ENST00000239243    142-201   
16        P43354  ENSG00000153234  ENST00000339562    260-335   
17        P78337  ENSG00000069011  ENST00000265340     89-148   
18        Q02548  ENSG00000196092  ENST00000358127     16-142   
19        Q03164  ENSG00000118058  ENST00000389506    169-180   
20        Q06413  ENSG00000081189  ENST00000437473      58-86   
21        Q06889  ENSG00000179388  ENST00000317216        NaN   
22        Q12857  ENSG00000162599  ENST00000403491      1-194   
23        Q13422  ENSG00000185811  ENST00000331340        NaN   
24        Q13485  ENSG00000141646  ENST00000342988        NaN   
25        Q14872  ENSG00000188786  ENST00000373036        NaN   
26        Q14938  ENSG00000008441  ENST00000592199      1-194   
27        Q15788  ENSG00000084676  ENST00000348332        NaN   
28        Q5T1R4  ENSG00000127124  ENST00000372583        NaN   
29        Q6N021  ENSG00000168769  ENST00000380013        NaN   
30        Q6P1N0  ENSG00000132024  ENST00000318003        NaN   
31        Q6ZRS2  ENSG00000080603  ENST00000262518  2857-2869   
32        Q86V15  ENSG00000130940  ENST00000377022        NaN   
33        Q8NBF1  ENSG00000174332  ENST00000312233        NaN   
34        Q92731  ENSG00000140009  ENST00000341099    149-214   
35        Q96QS3  ENSG00000004848  ENST00000379044    328-387   
36        Q9BXK1  ENSG00000129911  ENST00000250916        NaN   
37        Q9H4W6  ENSG00000108001  ENST00000355311        NaN   
38        Q9HBZ2  ENSG00000172379  ENST00000303329        NaN   
39        Q9NQB0  ENSG00000148737  ENST00000355995    350-418   
40        Q9UGL1  ENSG00000117139  ENST00000367265        NaN   
41        Q9UGU0  ENSG00000100207  ENST00000359486  1537-1551   
42        Q9UL68  ENSG00000186487  ENST00000428368        NaN   
43        Q9Y458  ENSG00000122145  ENST00000373294     96-283   
44        Q9Y4A8  ENSG00000050344  ENST00000056233        NaN   

                  AD_coords RD_coords Bif_coords length  
0                   340-477                           1  
1              199-263,2-91                           1  
2                     2-101                           1  
3           285-468,472-581                           1  
4                   220-273                           1  
5                      1-52                           1  
6           433-479,118-261                           1  
7           415-427,195-238                           1  
8             1-100,340-400                           1  
9     212-333,472-575,1-127                           1  
10                    8-285                           1  
11             1-69,371-397                           1  
12                  271-422                           1  
13                  172-354                           1  
14                  272-385                           1  
15                  234-267                           1  
16             1-91,584-59

In [28]:
TF_table_formatted = TF_table_formatted[["1","2", "uniprotID", "ENSG", "ENST", "DBD_coords", "AD_coords", "RD_coords", "Bif_coords", "length"]]
TF_table_formatted["DBD_coords"] = TF_table_formatted["DBD_coords"].fillna("NA-NA")
TF_table_formatted

1 2 uniprotID             ENSG             ENST DBD_coords  \
0         O14770  ENSG00000134138  ENST00000561208    276-338   
1         O60479  ENSG00000064195  ENST00000434704    129-188   
2         O75840  ENSG00000118263  ENST00000309446      NA-NA   
3         O94983  ENSG00000108509  ENST00000348066     30-155   
4         O95096  ENSG00000125820  ENST00000377142    128-187   
5         P10827  ENSG00000126351  ENST00000264637     53-127   
6         P11308  ENSG00000157554  ENST00000288319    311-391   
7         P11473  ENSG00000111424  ENST00000395324      21-96   
8         P15884  ENSG00000196628  ENST00000356073      NA-NA   
9         P19532  ENSG00000068323  ENST00000315869      NA-NA   
10        P20393  ENSG00000126368  ENST00000246672    129-205   
11        P25490  ENSG00000100811  ENST00000262238      NA-NA   
12        P26367  ENSG00000007372  ENST00000241001      4-130   
13        P32242  ENSG00000115507  ENST00000282549      38-97   
14        P35398  ENSG00000069667  ENST00000335670     73-138   
15        P35548  ENSG00000120149  ENST00000239243    142-201   
16        P43354  ENSG00000153234  ENST00000339562    260-335   
17        P78337  ENSG00000069011  ENST00000265340     89-148   
18        Q02548  ENSG00000196092  ENST00000358127     16-142   
19        Q03164  ENSG00000118058  ENST00000389506    169-180   
20        Q06413  ENSG00000081189  ENST00000437473      58-86   
21        Q06889  ENSG00000179388  ENST00000317216      NA-NA   
22        Q12857  ENSG00000162599  ENST00000403491      1-194   
23        Q13422  ENSG00000185811  ENST00000331340      NA-NA   
24        Q13485  ENSG00000141646  ENST00000342988      NA-NA   
25        Q14872  ENSG00000188786  ENST00000373036      NA-NA   
26        Q14938  ENSG00000008441  ENST00000592199      1-194   
27        Q15788  ENSG00000084676  ENST00000348332      NA-NA   
28        Q5T1R4  ENSG00000127124  ENST00000372583      NA-NA   
29        Q6N021  ENSG00000168769  ENST00000380013      NA-NA   
30        Q6P1N0  ENSG00000132024  ENST00000318003      NA-NA   
31        Q6ZRS2  ENSG00000080603  ENST00000262518  2857-2869   
32        Q86V15  ENSG00000130940  ENST00000377022      NA-NA   
33        Q8NBF1  ENSG00000174332  ENST00000312233      NA-NA   
34        Q92731  ENSG00000140009  ENST00000341099    149-214   
35        Q96QS3  ENSG00000004848  ENST00000379044    328-387   
36        Q9BXK1  ENSG00000129911  ENST00000250916      NA-NA   
37        Q9H4W6  ENSG00000108001  ENST00000355311      NA-NA   
38        Q9HBZ2  ENSG00000172379  ENST00000303329      NA-NA   
39        Q9NQB0  ENSG00000148737  ENST00000355995    350-418   
40        Q9UGL1  ENSG00000117139  ENST00000367265      NA-NA   
41        Q9UGU0  ENSG00000100207  ENST00000359486  1537-1551   
42        Q9UL68  ENSG00000186487  ENST00000428368      NA-NA   
43        Q9Y458  ENSG00000122145  ENST00000373294     96-283   
44        Q9Y4A8  ENSG00000050344  ENST00000056233      NA-NA   

                  AD_coords RD_coords Bif_coords length  
0                   340-477                           1  
1              199-263,2-91                           1  
2                     2-101                           1  
3           285-468,472-581                           1  
4                   220-273                           1  
5                      1-52                           1  
6           433-479,118-261                           1  
7           415-427,195-238                           1  
8             1-100,340-400                           1  
9     212-333,472-575,1-127                           1  
10                    8-285                           1  
11             1-69,371-397                           1  
12                  271-422                           1  
13                  172-354                           1  
14                  272-385                           1  
15                  234-267                           1  
16             1-91,584-59

In [29]:
TF_table_formatted.to_csv("../soto_analysis/outputs/TFs_table_proteins.txt", sep = "\t")